In [1]:
import pandas as pd
import numpy as np
import sys
import os
import pymysql
from sqlalchemy import create_engine
from datetime import datetime, date, timedelta

In [2]:
retail_df=pd.read_excel('零售司机活跃度.xlsx')
life_df=pd.read_excel('上海流量司机活跃度.xlsx')

In [9]:
retail_df.head()

,司机id,激活时间,准确激活时间,流量项目类型,安装时间,订单日期,激活周,有效周数,订单天数,周,weeknum,零售订单数
0,33,2018-04-14,2018-04-14,TS-0003,2018-09-09,2018-04-15,15.0,27.0,2.0,1.0,15.0,15
1,33,2018-04-14,2018-04-14,TS-0003,2018-09-09,2018-04-16,15.0,27.0,3.0,1.0,16.0,5
2,33,2018-04-14,2018-04-14,TS-0003,2018-09-09,2018-04-17,15.0,27.0,4.0,1.0,16.0,4
3,33,2018-04-14,2018-04-14,TS-0003,2018-09-09,2018-04-18,15.0,27.0,5.0,1.0,16.0,5
4,33,2018-04-14,2018-04-14,TS-0003,2018-09-09,2018-04-19,15.0,27.0,6.0,1.0,16.0,1


In [3]:
life_df.head()

,司机id,手机号,姓名,激活状态,流量编号,安装日期,订单日期,订单数,安装周,订单天数,订单周
0,47380,17858452743,朱耀文,1,TS-0002,2018-09-27,2018-09-27,3,39,1,1
1,47380,17858452743,朱耀文,1,TS-0002,2018-09-27,2018-10-12,1,39,16,3
2,47380,17858452743,朱耀文,1,TS-0002,2018-09-27,2018-10-18,3,39,22,4
3,116286,18301809970,张永,1,TS-0002,2018-11-02,2018-11-02,1,44,1,1
4,116401,17506176980,曹玉辉,-2,TS-0002,2018-09-28,2018-10-04,1,39,7,1


In [31]:
#零售每周激活司机数
retail_df.groupby(['激活周','司机id']).零售订单数.count().reset_index().groupby('激活周').司机id.count().reset_index()

,激活周,司机id
0,14.0,51
1,15.0,59
2,16.0,95
3,17.0,43
4,18.0,21
5,19.0,12
6,20.0,14
7,21.0,9
8,22.0,32
9,23.0,21


In [5]:
#小游戏每周安装司机数
life_df.groupby(['安装周','司机id']).订单数.count().reset_index().groupby('安装周').司机id.count().reset_index()

,安装周,司机id
0,39,140
1,41,71
2,42,75
3,43,36
4,44,17


In [32]:
#公众号每周安装司机数
life_df[life_df.流量项目类型=='TS-0003'].groupby(['安装周','司机id']).流量订单数.count().reset_index().groupby('安装周').司机id.count().reset_index()

,安装周,司机id
0,36,45
1,37,46
2,38,43
3,39,8
4,40,8
5,41,3


In [30]:
retail_df[retail_df.weeknum>=retail_df.激活周].groupby(['激活周','司机id','weeknum']).零售订单数.count().reset_index().pivot_table(index='激活周',
            columns='weeknum',values='零售订单数',aggfunc='count').loc[31.0:,31.0:]

weeknum,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0
激活周,,,,,,,,,,,,
31.0,25.0,19.0,16.0,14.0,13.0,13.0,11.0,12.0,12.0,10.0,9.0,NaN
32.0,NaN,32.0,30.0,25.0,21.0,22.0,19.0,18.0,18.0,13.0,14.0,NaN
33.0,NaN,NaN,14.0,14.0,11.0,9.0,11.0,9.0,10.0,8.0,6.0,1.0
34.0,NaN,NaN,NaN,17.0,15.0,11.0,11.0,9.0,10.0,11.0,9.0,NaN
35.0,NaN,NaN,NaN,NaN,9.0,9.0,7.0,9.0,8.0,10.0,6.0,1.0
36.0,NaN,NaN,NaN,NaN,NaN,15.0,17.0,15.0,10.0,12.0,9.0,NaN
37.0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,19.0,20.0,17.0,13.0,1.0
38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,5.0,4.0,5.0,NaN
39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,10.0,12.0,1.0


In [27]:
#小游戏司机留存
life_df[(life_df.weeknum>=life_df.安装周)&(life_df.流量项目类型=='TS-0002')].groupby(['安装周','司机id','weeknum']).\
流量订单数.count().reset_index().pivot_table(index='安装周',columns='weeknum',values='流量订单数',aggfunc='count')

weeknum,39,40,41,42
安装周,,,,
39,64.0,25.0,18.0,2.0
40,NaN,16.0,5.0,2.0
41,NaN,NaN,31.0,1.0
42,NaN,NaN,NaN,2.0


In [28]:
#公众号司机留存
life_df[(life_df.weeknum>=life_df.安装周)&(life_df.流量项目类型=='TS-0003')].groupby(['安装周','司机id','weeknum']).\
流量订单数.count().reset_index().pivot_table(index='安装周',columns='weeknum',values='流量订单数',aggfunc='count')

weeknum,36,37,38,39,40,41,42
安装周,,,,,,,
36,43.0,32.0,28.0,28.0,24.0,23.0,5.0
37,NaN,46.0,32.0,25.0,28.0,28.0,4.0
38,NaN,NaN,43.0,23.0,28.0,36.0,4.0
39,NaN,NaN,NaN,5.0,7.0,4.0,2.0
40,NaN,NaN,NaN,NaN,7.0,6.0,1.0
41,NaN,NaN,NaN,NaN,NaN,3.0,NaN
